# CNNs no TensorFlow

Neste notebook, vamos examinar como implementar uma rede neural convolucional no TensorFlow.

---
## Camadas convolucionais no TensorFlow

Nesta seção, vamos examinar como implementar uma camada convolucional no TensorFlow.

O TensorFlow fornece as funções [**`tf.nn.conv2d()`**](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d), [**`tf.nn.bias_add()`**](https://www.tensorflow.org/api_docs/python/tf/nn/bias_add) e [**`tf.nn.relu()`**](https://www.tensorflow.org/api_docs/python/tf/nn/relu) para que você possa criar suas próprias camadas convolucionais:

```
# Profundidade do output
k_output = 64

# Dimensões da imagem
img_width = 10
img_height = 10
color_channels = 3

# Filtro de convolução
filter_size_width = 5
filter_size_height = 5

# Peso e viés
weight = tf.Variable(tf.truncated_normal([filter_size_height, 
                                          filter_size_width, 
                                          color_channels, 
                                          k_output]))
bias = tf.Variable(tf.zeros(k_output))

# Aplicar convolução, adicionar viés e aplicar função de ativação
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], 
                          padding='SAME')
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)
```

O código acima usa a função [**`tf.nn.conv2d()`**](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) para computar a convolução usando `weight` como filtro e `[1, 2, 2, 1]` como passo.

É importante ressaltar que:

* O TensorFlow usa um passo para cada dimensão de entrada (`input`): `[batch, input_height, input_width, input_channels]`.
* Em geral, configuramos o passo para `batch` e `input_channels` (ou seja, o primeiro e quarto elementos do array `strides`) como `1`. Isso assegura que o modelo usa todos lotes e canais de entrada (*é uma boa prática remover do conjunto de dados lotes ou canais que você queira pular, em vez de usar o passo para pulá-los*).
* Você irá se concentrar em alterar `input_height` e `input_width` (mantendo `batch` e `input_channels` em 1). Os passos `input_height` e `input_width` servem para configurar a movimentação do filtro sobre a entrada `input`. O código exemplo usa um passo de 2 com um filtro 5x5 sobre `input`. Observe que o passo foi considerado um só número pois é simétrico, ou seja, `height = width`; quando alguém diz que está usando um passo de 2, isso significa que usou algo como `tf.nn.conv2d(x, W, stride=[1, 2, 2, 1])`.

A função [**`tf.nn.bias_add()`**](https://www.tensorflow.org/api_docs/python/tf/nn/bias_add) adiciona um viés unidimensional à última dimensão da matriz (observe que usar [**`tf.add`**](https://www.tensorflow.org/api_docs/python/tf/add) não funciona quando os tensores não são do mesmo formato).

A função [**`tf.nn.relu()`**](https://www.tensorflow.org/api_docs/python/tf/nn/relu) aplica uma função de ativação ReLu à camada.

### Usando camadas convolucionais no TensorFlow

Agora vamos construir uma camada convolucional no TensorFlow. Na célula abaixo, será solicitado que você configure as dimensões dos filtros de convolução, bem como os pesos e vieses. Esta é, de diversas formas, a parte mais complicada de se usar CNNs no TensorFlow. Quando você tiver uma noção de como configurar as dimensões destes atributos, usar CNNs será muito mais direto ao ponto.

Recomendamos que você reveja a documentação do TensorFlow para [**convoluções 2D**](https://www.tensorflow.org/api_guides/python/nn#Convolution). A maior parte da documentação é bem direta, exceto talvez a explicação do argumento `padding` (espaçamento), que pode variar dependendo de você paassar `'VALID'` ou `'SAME'`. Além disso, recomendamos que também revise sobre [**variáveis no TensorFlow**](https://www.tensorflow.org/programmers_guide/variables) e sobre como determinar as dimensões da saída baseado no tamanho da entrada e do filtro (você usará isso para determinar o tamanho que seu filtro deve ter).

In [ ]:
import tensorflow as tf
import numpy as np

# tf.nn.conv2d requer que o input seja 4D (batch_size, height, width, 
# depth)
x = np.array([[0, 1, 0.5, 10], 
              [2, 2.5, 1, -8], 
              [4, 0, 5, 6], 
              [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

# Configure strides, padding, pesos e vieses de modo que o formato da
# saída seja (1, 2, 2, 3)
def conv2d(input_tensor):
    # TODO: Definir os filtros weight e bias
    # O formato de weight é (height, width, input_depth, output_depth)
    F_W = tf.Variable(tf.truncated_normal((2, 2, 1, 3)))
    # O formato de bias é (output_depth,)
    F_b = tf.Variable(tf.zeros(3))

    
    # TODO: Configurar stride para cada dimensão (batch_size, height, 
    # width, depth)
    strides = [1, 2, 2, 1]
    
    # TODO: configurar padding como 'VALID' ou 'SAME'
    padding = 'VALID'
    
    # Como tf.nn.conv2d() não inclui bias, este deve ser adicionado
    # manualmente
    no_bias = tf.nn.conv2d(input_tensor, F_W, strides, padding)
    output = tf.nn.bias_add(no_bias, F_b)
    return output

output_tensor = conv2d(X)

In [ ]:
print(output_tensor.shape)

---
## Camadas maxpooling no TensorFlow

Conceitualmente, o benefício da operação de [agrupamento máximo](https://en.wikipedia.org/wiki/Convolutional_neural_network#Pooling_layer) é a redução do tamanho de entrada, permitindo ue a rede neural se concentre apenas nos elementos mais importantes. O agrupamento máximo faz isso retendo apenas o valor máximo para cada área filtrada, e removendo os valores remanescentes.

O TensorFlow fornece a função [**`tf.nn.max_pool()`**](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool) para aplicar a operação de agrupamento máximo às suas camadas convolucionais.

```
# Aplicar maxpooling
conv_layer = tf.nn.max_pool(conv_layer, 
                            ksize=[1, 2, 2, 1], 
                            strides=[1, 2, 2, 1], 
                            padding='SAME')
```

A função [**`tf.nn.max_pool()`**](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool) aplica o agrupamento máximo com o parâmetro `ksize` como tamanho do filtro e o parâmetro `strides` como o tamanho do passo. Filtros 2x2 com um passo 2x2 são comuns, na prática.

Esses dois parâmetros, `ksize` e `strides`, são estruturados como listas de quatro elementos, cada um correspondente à uma dimensão do tensor de entrada(`[batch, height, width, channels]`). Para ambos, as dimensões `batch` e `channels` são tipicamente configuradas como `1`.

### Usando camadas maxpooling no TensorFlow

Na célula abaixo, será solicitado que você configure as dimensões dos filtros de agrupamento, passos e espaçamento. 

Recomendamos que faça a leitura da documentação do TensorFlow para [**`tf.nn.max_pool()`**](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool). Note que o espaçamento (padding) funciona da mesma forma como na convolução.

In [ ]:
import tensorflow as tf
import numpy as np

# tf.nn.max_pool requer que o input seja 4D (batch_size, height, width, 
# depth)
x = np.array([[0, 1, 0.5, 10], 
              [2, 2.5, 1, -8], 
              [4, 0, 5, 6], 
              [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

# Configure strides, padding e ksize de modo que o formato da saída 
# seja (1, 2, 2, 1)
def maxpool(input_tensor):
    # TODO: Definir o filtro ksize
    # O formato de ksize é (height, width, input_depth, output_depth)
    ksize = [1, 2, 2, 1]
    
    # TODO: Configurar stride para cada dimensão (batch_size, height, 
    # width, depth)
    strides = [1, 2, 2, 1]
    
    # TODO: configurar padding como 'VALID' ou 'SAME'
    padding = 'VALID'
    
    output = tf.nn.max_pool(input_tensor, ksize, strides, padding)
    return output

output_tensor = maxpool(X)

In [ ]:
print(output_tensor.shape)

---
## Redes neurais convolucionais no TensorFlow

Agora é hora de mostrarmos um exemplo passo a passo de rede neural convolucional no TensorFlow. A estrutura do código que vamos usar segue uma estrutura clássica de CNNs, que é uma mistura de camadas convolucionais de agrupamento máximo, seguidas por camadas completamente conectadas. O código pe similar ao que vimos [neste notebook](https://github.com/vilacham/tensorflow_introduction/blob/master/redes_neurais_profundas.ipynb), exceto pelo fato de que reestruturamos a arquitetura da rede como uma CNN.

### O conjunto de dados

Já vimos esta seção de código. Aqui, vamos importar o conjunto de dados do MNIST e usar uma função conveniente do TensorFlow para organizá-lo em lotes, escalar e aplicar a codificação one-hot aos dados.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.', one_hot=True, reshape=False)

### Definindo hiperparâmetros

Aqui, vamos configurar os hiperparâmetros da CNN: taxa de aprendizado, número de épocas, tamanho dos lotes, quantidade de classes, etc.

In [ ]:
learning_rate = 0.00001    # Taxa de aprendizado
epochs = 100               # Número de épocas
batch_size = 128           # Tamanho dos lotes
test_valid_size = 256      # Número de amostras para calcular validação e acurácia
n_classes = 10             # Quantidade de classes
dropout = 0.75             # Probabilidade de manter as unidades

### Pesos e vieses

Abaixo, serão criadas listas para armazenar os pesos e vieses. Nossa CNN contará com duas camadas alternando entre convoluções e agrupamento máximo, seguidas por uma camada completamente conectada e uma camada de saída.

In [ ]:
import tensorflow as tf

weights = {'w_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
           'w_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
           'w_fc': tf.Variable(tf.random_normal([7*7*64, 1024])), 
           'w_out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {'b_conv1': tf.Variable(tf.random_normal([32])), 
          'b_conv2': tf.Variable(tf.random_normal([64])), 
          'b_fc': tf.Variable(tf.random_normal([1024])), 
          'b_out': tf.Variable(tf.random_normal([n_classes]))}

### Camadas convolucionais

Usaremos as funções [**`tf.nn.conv2d()`**](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d), [**`tf.nn.bias_add()`**](https://www.tensorflow.org/api_docs/python/tf/nn/bias_add) e [**`tf.add`**](https://www.tensorflow.org/api_docs/python/tf/add) para definir uma função que computa a convolução usando pesos e adicionando o viés, e em seguida aplica uma função de ativação ReLU.

In [ ]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], 
                     padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

### Camadas de maxpooling

Usaremos a função [**`tf.nn.max_pool()`**](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool) para dfinir uma função que aplica o agrupamento máximo a cada camada `x` usando um filtro de tamanho `k`.

In [ ]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], 
                          padding='SAME')

### Modelo

Na célula abaixo vamos definir uma função que cria o nosso modelo.

In [ ]:
def conv_net(x, weights, biases, dropout):
    # Camada 1 (convolucional + maxpooling) - 28x28x1 para 14x14x32
    conv1 = conv2d(x, weights['w_conv1'], biases['b_conv1'])
    conv2 = maxpool2d(conv1, k=2)
    
    # Camada 2 (convolucional + maxpooling) - 14x14x32 para 7x7x64
    conv2 = conv2d(conv1, weights['w_conv2'], biases['b_conv2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # Camada 3 (completamente conectada) - 7x7x64 para 1032
    fc1 = tf.reshape(conv2, [-1, weights['w_fc'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['w_fc']), biases['b_fc'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Camada de saída (previsão de cada classe)
    output = tf.add(tf.matmul(fc1, weights['w_out']), biases['b_out'])
    return output

### Treinamento

Agora vamos treinar nossa CNN.

In [ ]:
# Grafo
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Modelo
logits = conv_net(x, weights, biases, keep_prob)

# Função de custo e otimizador
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
optmizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Acurácia
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Inicializador de variáveis
init = tf.global_variables_initializer()

# Iniciar sessão
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optmizer, feed_dict={x: batch_x, 
                                          y: batch_y, 
                                          keep_prob: dropout})
            loss = sess.run(cost, feed_dict={x: batch_x, 
                                             y: batch_y, 
                                             keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={x: mnist.validation.images[:test_valid_size], 
                                                      y: mnist.validation.labels[:test_valid_size], 
                                                      keep_prob: 1.})
            print('Epoch {:>2}, Batch {:>3}: Loss = {:>10.4f} | Validation accuracy = {:.6f}'.format(epoch + 1, batch + 1, loss, valid_acc))

    # Calcular acurácia no conjunto de teste
    test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images[:test_valid_size], 
                                             y: mnist.test.labels[:test_valid_size], 
                                             keep_prob: 1.})
    print('Test accuracy: {:.6f}'.format(test_acc))